# WMI Persistence
Detects suspicious WMI event filter and command line event consumer based on event id 5861 and 5859 (Windows 10, 2012 and higher)

## Rule Content
```
- title: WMI Persistence
  id: 0b7889b4-5577-4521-a60a-3376ee7f9f7b
  status: experimental
  description: Detects suspicious WMI event filter and command line event consumer
    based on event id 5861 and 5859 (Windows 10, 2012 and higher)
  author: Florian Roth
  references:
  - https://twitter.com/mattifestation/status/899646620148539397
  - https://www.eideon.com/2018-03-02-THL03-WMIBackdoors/
  tags:
  - attack.execution
  - attack.persistence
  - attack.t1047
  logsource:
    product: windows
    service: wmi
    category: null
  detection:
    selection:
      EventID: 5861
    keywords:
      Message:
      - '*ActiveScriptEventConsumer*'
      - '*CommandLineEventConsumer*'
      - '*CommandLineTemplate*'
    selection2:
      EventID: 5859
    condition: selection and 1 of keywords or selection2
  falsepositives:
  - Unknown (data set is too small; further testing needed)
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-wmiactivity-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"5861" AND Message.keyword:(*ActiveScriptEventConsumer* OR *CommandLineEventConsumer* OR *CommandLineTemplate*)) OR event_id:"5859")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()